In [6]:
import requests
import os

def download_nq_dataset(destination_dir):
    # URL for the dataset
    url = "https://storage.googleapis.com/natural_questions/v1.0-simplified/simplified-nq-train.jsonl.gz"
    
    # Make sure the destination directory exists
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # Path to save the file
    save_path = os.path.join(destination_dir, "simplified-nq-train.jsonl.gz")
    
    # Download the file
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raise an error for failed requests
    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)

    print(f"Downloaded the dataset to {save_path}")

# Example usage
destination_dir = "../data/natural_questions/"  # or any other directory you prefer
download_nq_dataset(destination_dir)


Downloaded the dataset to ../data/natural_questions/simplified-nq-train.jsonl.gz


In [10]:
import gzip
import json

data = []

with gzip.open('../data/natural_questions/simplified-nq-train.jsonl.gz', 'rt', encoding='utf-8') as f:
    for i, line in enumerate(f):
        entry = json.loads(line)
        print(entry['question_text'])
        if i > 50:
            break
        data.append(entry)


which is the most common use of opt-in e-mail marketing
how i.met your mother who is the mother
what type of fertilisation takes place in humans
who had the most wins in the nfl
what happened to the lost settlement of roanoke
what are the different regions of africa and how do they differ
who played mantis guardians of the galaxy 2
who did the voice of the magician in frosty the snowman
what indian tribe did the acadians form friendships and alliances with
what is considered the outer banks in north carolina
who is deputy cm of j and k
bangko sentral ng pilipinas (central bank of the philippines)
actual time taken from start to finish to produce one unit of value
i had trouble in getting to solla sellew
ed sheeran i ' m in love with your body
the nashville sound brought a polished and cosmopolitan sound to country music by
the new atlantis a journal of technology and society
list the seven wonders of the modern world
rolling stone top bands of all time list
what channel is the premier 